In [1]:
import numpy as np
import pandas as pd
import time
import ast
from tqdm import tqdm
import datetime
from multiprocessing import Pool, cpu_count
from itertools import zip_longest
from collections import defaultdict, OrderedDict, Counter
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import chi2,SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from scipy import stats
from scipy.stats import skew, norm
from itertools import product
%matplotlib inline

In [4]:
base_dir = './dataset/'

### 1. user_app_usage.csv 生成统计量数据

In [5]:
user_app_usage_hot=pd.read_csv(base_dir+'user_app_usage.csv',header=None,names=['uId','appId','duration','times','use_date'],
                           dtype={'uId':int,'duration':np.uint32,'times':np.uint32,'use_date':'category'})
print(user_app_usage_hot.shape)

(840560515, 5)


In [6]:
print(user_app_usage_hot.head())

        uId    appId  duration  times    use_date
0  20765863  a003838      3012     67  2019-07-16
1  20765863  a003838      1670     11  2019-06-28
2  20765863  a003838       390     29  2019-07-08
3  20765863  a003838      1793     12  2019-07-24
4  20765863  a003838      6905     33  2019-07-06


In [7]:
user_app_usage_hot['use_date'] = pd.to_datetime(user_app_usage_hot['use_date'])
user_app_usage_hot['dayofweek'] = user_app_usage_hot['use_date'].dt.dayofweek
user_app_usage_hot['dayofweek'] = user_app_usage_hot['dayofweek'].astype(np.uint8)

In [8]:
# 每天所有app使用时长总和、总次数
dtime = user_app_usage_hot.groupby(['uId', 'use_date'])['duration','times'].agg('sum')
dtime = dtime.reset_index()
dtime.head()

,uId,use_date,duration,times
0,1000009,2019-06-27,20387.0,644.0
1,1000009,2019-06-28,6938.0,435.0
2,1000009,2019-06-29,20519.0,1205.0
3,1000009,2019-06-30,20362.0,1194.0
4,1000009,2019-07-01,14531.0,709.0


In [9]:
# 一个月内app使用情况统计量
dtime_stat = dtime.groupby(['uId'])['duration','times'].agg(
    {'sum': 'sum', 'mean': 'mean', 'std': 'std', 'max': 'max'}).reset_index()
dtime_stat.columns = ['uId','max_duration','max_times','mean_duration','mean_times','sum_durations','sum_times',
                     'std_durations','std_times']
dtime_stat.shape

/opt/conda/lib/python3.6/site-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


(4020281, 9)

In [10]:
dtime_stat.head()

,uId,max_duration,max_times,mean_duration,mean_times,sum_durations,sum_times,std_durations,std_times
0,1000009,576119.0,20342.0,19203.966667,678.066667,6927.221215,267.930760,35001.0,1634.0
1,1000010,557640.0,22889.0,18588.000000,762.966667,4542.816125,177.772905,28655.0,1185.0
2,1000011,130512.0,6904.0,4350.400000,230.133333,2356.967303,111.743158,11502.0,479.0
3,1000012,344999.0,13484.0,11499.966667,449.466667,3897.897492,199.559469,19451.0,1198.0
4,1000014,358821.0,304.0,18885.315789,16.000000,34719.077708,15.627611,86400.0,67.0


In [11]:
# 每周7天每天app使用时长与次数总和
wtime = user_app_usage_hot.groupby(['uId', 'dayofweek'])['duration','times'].agg('sum')
wtime = wtime.reset_index()
wtime.head()

,uId,dayofweek,duration,times
0,1000009,0,73331,2384
1,1000009,1,64721,2094
2,1000009,2,65775,2420
3,1000009,3,99700,3206
4,1000009,4,90344,3364


In [12]:
weektime = wtime.pivot(
    index='uId', columns='dayofweek', values=['duration','times']).fillna(0)
weektime.reset_index(inplace=True)
weektime.columns = ['uId','w0_duration','w1_duration','w2_duration','w3_duration','w4_duration','w5_duration','w6_duration',
                   'w0_times','w1_times','w2_times','w3_times','w4_times','w5_times','w6_times']
weektime.shape

(4020281, 15)

In [13]:
weektime.head()

,uId,w0_duration,w1_duration,w2_duration,w3_duration,w4_duration,w5_duration,w6_duration,w0_times,w1_times,w2_times,w3_times,w4_times,w5_times,w6_times
0,1000009,73331.0,64721.0,65775.0,99700.0,90344.0,98089.0,84159.0,2384.0,2094.0,2420.0,3206.0,3364.0,3922.0,2952.0
1,1000010,74085.0,73888.0,70744.0,93065.0,94658.0,76681.0,74519.0,3332.0,3078.0,2806.0,3297.0,3814.0,3319.0,3243.0
2,1000011,11039.0,16031.0,17954.0,21421.0,23447.0,16705.0,23915.0,591.0,788.0,988.0,1089.0,1303.0,819.0,1326.0
3,1000012,36020.0,49517.0,45441.0,69273.0,59790.0,45581.0,39377.0,1689.0,2082.0,1852.0,2946.0,2657.0,1180.0,1078.0
4,1000014,88813.0,2472.0,86475.0,166077.0,7154.0,5834.0,1996.0,21.0,29.0,13.0,118.0,71.0,32.0,20.0


In [19]:
# 每人每天使用的app列表--------较耗时
dapp = user_app_usage_hot[['uId', 'use_date', 'appId']].drop_duplicates().groupby(
    ['uId', 'use_date'])['appId'].agg(' '.join)
dapp = dapp.reset_index()
dapp.head()

,uId,use_date,appId
0,1000009,2019-06-27,a002392 a003386 a007319 a007439 a001015 a00836...
1,1000009,2019-06-28,a002392 a003386 a003659 a004078 a007439 a00934...
2,1000009,2019-06-29,a002392 a003386 a003659 a004078 a007439 a00731...
3,1000009,2019-06-30,a002392 a003386 a007319 a007439 a006515 a00845...
4,1000009,2019-07-01,a002392 a003386 a005480 a007439 a006515 a00845...


In [47]:
list(dapp.loc[(dapp['uId']==1000009)&(dapp['use_date']=='2019-06-27'),'appId'])

['a002392 a003386 a007319 a007439 a001015 a008365 a009346 a005128 a007087 a00838 a004076 a007331 a008879 a001062 a007104']

In [48]:
list(user_app_usage_hot.loc[(user_app_usage_hot['uId']==1000009)&(user_app_usage_hot['use_date']=='2019-06-27'),'appId'])

['a002392',
 'a003386',
 'a007319',
 'a007439',
 'a001015',
 'a008365',
 'a009346',
 'a005128',
 'a007087',
 'a00838',
 'a004076',
 'a007331',
 'a008879',
 'a001062',
 'a007104']

In [26]:
dapp.shape

(120608430, 3)

In [35]:
dapp[dapp['appId'].isna()].shape

(13858552, 3)

In [36]:
dapp.dropna(inplace=True)

In [37]:
dapp.shape

(106749878, 3)

In [38]:
# 每人一个月内使用的app数量统计
dapp['app_len'] = dapp['appId'].apply(lambda x: x.split(' ')).apply(len)
dapp_stat = dapp.groupby('uId')['app_len'].agg(
    {'std': 'std', 'mean': 'mean', 'max': 'max'})
dapp_stat = dapp_stat.reset_index()
dapp_stat.columns = ['uId', 'app_len_max', 'app_len_mean', 'app_len_std']
dapp_stat.shape

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  after removing the cwd from sys.path.


(4020281, 4)

In [39]:
dapp_stat.head()

,uId,app_len_max,app_len_mean,app_len_std
0,1000009,2.918234,13.966667,20
1,1000010,4.568885,19.433333,29
2,1000011,1.003442,2.400000,4
3,1000012,1.774986,6.766667,10
4,1000014,0.904828,2.473684,4


In [40]:
app_usage_stat_df = pd.merge(dapp_stat, dtime_stat, on='uId', how='left')
app_usage_stat_df = app_usage_stat_df.merge(weektime, on='uId',how='left')
app_usage_stat_df.shape

(4020281, 26)

In [41]:
train_id_set = pd.read_csv(base_dir+'age_train.csv',header=None, names=['uId', 'age_group'])
test_id_set = pd.read_csv(base_dir+'age_test.csv', header=None, names=['uId'])
print(test_id_set.shape,train_id_set.shape)

(1000000, 1) (4000000, 2)


In [42]:
train_app_usage_stat_df = train_id_set[['uId']].merge(app_usage_stat_df,on='uId',how='left')
train_app_usage_stat_df.fillna(0, inplace=True)
test_app_usage_stat_df = test_id_set[['uId']].merge(app_usage_stat_df,on='uId',how='left')
test_app_usage_stat_df.fillna(0, inplace=True)
print(test_app_usage_stat_df.shape,train_app_usage_stat_df.shape)

(1000000, 26) (4000000, 26)


In [43]:
h5 = pd.HDFStore(base_dir+'train_app_usage_stat_df.h5','w',complevel=4,complib='blosc')
h5['data'] = train_app_usage_stat_df
h5.close()

In [44]:
h5 = pd.HDFStore(base_dir+'test_app_usage_stat_df.h5','w',complevel=4,complib='blosc')
h5['data'] = test_app_usage_stat_df
h5.close()

In [16]:
print(0)

0
